In [67]:
!pip install -qU \
  langchain-core \
  langchain-openai \
  langchain-community \
  langsmith \
  


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [68]:
!pip install -qU "langchain[google-genai]"




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [69]:
import os
from getpass import getpass
from dotenv import load_dotenv


# must enter API key

load_dotenv()
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
if not langchain_api_key:
    raise ValueError("LANGCHAIN_API_KEY is not set")

os.environ["LANGCHAIN_API_KEY"] = langchain_api_key



# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

#project name as preffered
os.environ["LANGCHAIN_PROJECT"] = "First agent"

In [70]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return (x + y)

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return (y - x)

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return (x * y)

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return (x ** y)

In [71]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x112c21a80>)

In [72]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [73]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [74]:
# CREATING AN AGENT USING LANGCHAIN'S LCEL

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


In [75]:
from langchain.chat_models import init_chat_model

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY is not set")

os.environ["GEMINI_API_KEY"] = api_key
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [76]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

agent: RunnableSerializable = (
    {
        "input" : lambda x: x["input"],
        "chat_history" : lambda x: x["chat_history"],
        "agent_scratchpad" : lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [77]:
tool_call = agent.invoke({"input" : "What is 6 * 7", "chat_history" : []})
tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 7.0, "x": 6.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--a4b0e7a7-29d7-47da-9bba-602559c319ac-0', tool_calls=[{'name': 'multiply', 'args': {'y': 7.0, 'x': 6.0}, 'id': '583add9d-fe16-4155-af33-354004114905', 'type': 'tool_call'}], usage_metadata={'input_tokens': 274, 'output_tokens': 118, 'total_tokens': 392, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 100}})

In [78]:
tool_call.tool_calls

[{'name': 'multiply',
  'args': {'y': 7.0, 'x': 6.0},
  'id': '583add9d-fe16-4155-af33-354004114905',
  'type': 'tool_call'}]

In [79]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

In [80]:
tools = [final_answer, add, subtract, multiply, exponentiate]

# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

name2tool

{'final_answer': <function __main__.final_answer(answer: str, tools_used: list[str]) -> str>,
 'add': <function __main__.add(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>}

In [81]:
tool_call = agent.invoke({"input": "What is 6 * 7", "chat_history": []})
tool_call.tool_calls

[{'name': 'multiply',
  'args': {'y': 7.0, 'x': 6.0},
  'id': '12a65d8c-512d-403c-88fd-5640ee56c177',
  'type': 'tool_call'}]

In [82]:
from langchain_core.messages import ToolMessage

tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 6 * 7",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='', additional_kwargs={'function_call': {'name': 'final_answer', 'arguments': '{"tools_used": ["multiply"], "answer": "6 multiplied by 7 is 42."}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--b25e9de1-3d2e-4a41-98d3-b03c0a697685-0', tool_calls=[{'name': 'final_answer', 'args': {'tools_used': ['multiply'], 'answer': '6 multiplied by 7 is 42.'}, 'id': 'e7cb9895-4e58-41eb-bfaf-58bc44fc7090', 'type': 'tool_call'}], usage_metadata={'input_tokens': 417, 'output_tokens': 110, 'total_tokens': 527, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 80}})

In [83]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'tools_used': ['multiply'], 'answer': '6 multiplied by 7 is 42.'},
  'id': 'e7cb9895-4e58-41eb-bfaf-58bc44fc7090',
  'type': 'tool_call'}]